In [1]:
import requests
import urllib
import pickle
import os
import json
import time

In [2]:
from multiprocessing import Pool

In [3]:
class MSFaceAPIClient():
    def __init__(self, subscription_key, location):
        self._subscription_key = subscription_key
        self._location = location
        
    def face_detect_local(self, image_path):
        headers = {
            # Request headers
            'Content-Type': 'application/octet-stream',
            'Ocp-Apim-Subscription-Key': self._subscription_key,
        }

        params = urllib.parse.urlencode({
            # Request parameters
            'returnFaceId': 'true',
            'returnFaceLandmarks': 'true',
            'returnFaceAttributes': "age,gender,headPose,smile,facialHair,glasses,emotion,hair,makeup,occlusion,accessories,blur,exposure,noise"
        })

        try:
            url = "https://" + self._location + ".api.cognitive.microsoft.com/face/v1.0/detect"
            return requests.post(url, params=params, headers=headers, data=open(image_path, 'rb')).json()
        except Exception as e:
            print("Detect Face Error: " + str(e))
            return None

In [ ]:
# Run to delete error files
out_path = "/mnt/e/Data/selfies/face_api_json"
    
shortcodes_analysed = os.listdir(out_path)
    
for shortcode in shortcodes_analysed:
    if os.path.exists(os.path.join(out_path, shortcode)):
         with open(os.path.join(out_path, shortcode), "r") as fin:
            data = json.load(fin)
            if "error" in data:
                print("Deleting: " + shortcode)
                os.remove(os.path.join(out_path, shortcode))

In [4]:
face_api_client = MSFaceAPIClient(
    subscription_key = "1f5456936f2e4369b2226440b624dd00",
    location = "westcentralus"
)

in_path = "/mnt/e/Data/selfies/images"
out_path = "/mnt/e/Data/selfies/face_api_json"

shortcodes_analysed_json = os.listdir(in_path)
shortcodes_analysed = list(map(lambda x: x[:-4], shortcodes_analysed_json))

for shortcode in shortcodes_analysed:
    if not os.path.exists(os.path.join(out_path, shortcode + ".json")):
        image_path = os.path.join(in_path, shortcode + ".jpg")
        data = face_api_client.face_detect_local(image_path)
        time.sleep(1)
        if data is not None:
            if "error" not in data:
                with open(os.path.join(out_path, shortcode + ".json"), "w") as fout:
                    json.dump(data, fout)
            else:
                print(data["error"])
                time.sleep(5)